In [ ]:
import pdfplumber
import docx
import os
import time
import json # For saving results
import numpy as np
from google.colab import drive
from keybert import KeyBERT
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch # To check for GPU availability
import nltk
import re # <-- TAMBAHAN: Untuk Regular Expressions (mencari referensi)

# --- Global Settings & Configuration ---
# [cite_start]Path ke folder dataset Anda di Google Drive [cite: 323, 330]
DATASET_FOLDER = "/content/drive/MyDrive/paper_TA"
# Path untuk menyimpan hasil JSON
RESULTS_JSON_PATH = "/content/drive/MyDrive/PaperTA_results.json"
GRAPH_JSON_PATH = "/content/drive/MyDrive/PaperTA_graph_data.json"
SENTENCE_RELATIONS_JSON_PATH = "/content/drive/MyDrive/PaperTA_sentence_relations.json"

# Batas karakter untuk input model (pencegahan error GPU/memori)
# Sesuaikan jika perlu, 4000 char ~ 1000-1200 tokens
MAX_CHARS_FOR_MODEL = 3000
# Ambang batas kemiripan untuk visualisasi hubungan
SIMILARITY_THRESHOLD = 0.50
SENTENCE_SIMILARITY_THRESHOLD = 0.50

NLTK_DATA_PATH = "/content/nltk_data"
os.makedirs(NLTK_DATA_PATH, exist_ok=True)

if NLTK_DATA_PATH not in nltk.data.path:
    nltk.data.path.append(NLTK_DATA_PATH)
print(f"NLTK Data Path: {nltk.data.path}")

print(f"\nMemastikan data NLTK (punkt) ada di {NLTK_DATA_PATH}...")
try:
    # Coba cari di path spesifik kita
    nltk.data.find('tokenizers/punkt', paths=[NLTK_DATA_PATH])
    print("✅ Data NLTK (punkt) sudah tersedia di lokasi kustom.")
except LookupError:
    print(f"⚠ Data NLTK (punkt) tidak ditemukan, mencoba mengunduh ke {NLTK_DATA_PATH}...")
    try:
        nltk.download('punkt', download_dir=NLTK_DATA_PATH, quiet=True) # Download ke folder spesifik
        # Verifikasi ulang setelah download
        nltk.data.find('tokenizers/punkt', paths=[NLTK_DATA_PATH])
        print("✅ Data NLTK (punkt) berhasil diunduh dan diverifikasi.")
    except Exception as download_error:
        print(f"❌ FATAL ERROR: Gagal mengunduh data NLTK (punkt). Error: {download_error}")
        raise download_error # Hentikan jika download gagal

# Cek ketersediaan GPU
DEVICE = 0 if torch.cuda.is_available() else -1 # 0 for first GPU, -1 for CPU
DEVICE_NAME = torch.cuda.get_device_name(0) if DEVICE == 0 else "CPU"
print(f"💡 Menggunakan Perangkat: {DEVICE_NAME}")

# --- Mount Google Drive ---
print("\nMenghubungkan Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True) # force_remount helps avoid stale mounts
    print("✅ Google Drive terhubung.")
    # Verifikasi path dataset
    if not os.path.exists(DATASET_FOLDER):
        print(f"⚠ Peringatan: Folder dataset '{DATASET_FOLDER}' tidak ditemukan di Google Drive Anda.")
    else:
        print(f"✅ Folder dataset ditemukan di '{DATASET_FOLDER}'.")
except Exception as e:
    print(f"❌ Gagal menghubungkan Google Drive: {e}")

NLTK Data Path: ['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '/content/nltk_data']

Memastikan data NLTK (punkt) ada di /content/nltk_data...
✅ Data NLTK (punkt) sudah tersedia di lokasi kustom.
💡 Menggunakan Perangkat: Tesla T4

Menghubungkan Google Drive...
Mounted at /content/drive
✅ Google Drive terhubung.
✅ Folder dataset ditemukan di '/content/drive/MyDrive/paper_TA'.


In [ ]:
import nltk # Pastikan diimpor
import os # Pastikan diimpor
import pdfplumber # Pastikan diimpor
import docx # Pastikan diimpor
import re # Pastikan diimpor

NLTK_DATA_PATH = "/content/nltk_data"
os.makedirs(NLTK_DATA_PATH, exist_ok=True)

def split_into_sentences(text):
    """Memisahkan teks panjang menjadi kalimat menggunakan NLTK,
       dengan MEMAKSA download/verifikasi punkt SETIAP KALI."""
    if not text or not isinstance(text, str):
        return []

    # --- LANGKAH KRUSIAL: PASTIKAN PUNKT DIUNDUH/ADA DI SINI ---
    try:
        #print("DEBUG: Memaksa download punkt...") # Uncomment jika perlu debug
        nltk.download('punkt', download_dir=NLTK_DATA_PATH, quiet=True, force=True, raise_on_error=True)
        # Pastikan path dikenali (meskipun seharusnya sudah)
        if NLTK_DATA_PATH not in nltk.data.path:
            nltk.data.path.append(NLTK_DATA_PATH)
        # print("DEBUG: Download selesai, path:", nltk.data.path) # Uncomment jika perlu debug
    except Exception as download_error:
        print(f"❌ FATAL ERROR: Gagal download paksa punkt. Error: {download_error}")
        return [] # Kembalikan list kosong
    # -----------------------------------------------

    # --- Panggil sent_tokenize ---
    try:
        # Panggil setelah download paksa
        sentences = nltk.sent_tokenize(text)
        return [s.strip() for s in sentences if len(s.strip()) > 10]
    except LookupError as le:
         # Jika MASIH LookupError, ini sangat aneh
         print(f"❌ Error LookupError saat sent_tokenize MESKIPUN sudah download paksa: {le}")
         print(f"   NLTK mencari di: {nltk.data.path}")
         # Coba verifikasi manual file yang dicari
         try:
             finder = nltk.data.find('tokenizers/punkt', paths=[NLTK_DATA_PATH])
             print(f"   VERIFIKASI MANUAL: Ditemukan di {finder}")
         except LookupError:
             print(f"   VERIFIKASI MANUAL: Tetap tidak ditemukan di {NLTK_DATA_PATH}.")
         return []
    except Exception as e:
        print(f"❌ Error lain saat memanggil nltk.sent_tokenize: {e}")
        return []

def extract_text_from_pdf(file_path):
    """Mengekstrak teks mentah dari file PDF, dengan pembersihan dasar."""
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                # x_tolerance=1 mengurangi spasi antar kata yang salah gabung
                # y_tolerance=3 membantu menggabungkan paragraf yang terpisah dekat
                page_text = page.extract_text(x_tolerance=1, y_tolerance=3)
                if page_text:
                    text += page_text + "\n"
        # Pembersihan: hapus baris baru berlebih & spasi di awal/akhir baris
        cleaned_lines = [line.strip() for line in text.splitlines() if line.strip()]
        return "\n".join(cleaned_lines)
    except Exception as e:
        print(f"⚠ Gagal memproses PDF {os.path.basename(file_path)}: {e}")
        return None

def extract_text_from_docx(file_path):
    """Mengekstrak teks mentah dari file DOCX, dengan pembersihan dasar."""
    text = ""
    try:
        doc = docx.Document(file_path)
        paragraphs = [para.text.strip() for para in doc.paragraphs if para.text.strip()]
        return "\n\n".join(paragraphs) # Beri jarak antar paragraf
    except Exception as e:
        print(f"⚠ Gagal memproses DOCX {os.path.basename(file_path)}: {e}")
        return None

# --- FUNGSI BARU UNTUK EKSTRAKSI REFERENSI ---
def extract_references(full_text):
    """
    Mencoba mengekstrak daftar referensi dari teks lengkap menggunakan regex.
    """
    if not full_text:
        return []

    # Pola regex untuk menemukan awal bagian referensi
    # Mencari di awal baris (^) dengan spasi opsional (\s*)
    # \b memastikan kita mencocokkan kata utuh (misal "References" bukan "Sub-references")
    pattern = re.compile(r'^\s*(\bREFERENCES\b|\bDAFTAR PUSTAKA\b|\bBIBLIOGRAPHY\b)\s*$',
                         re.IGNORECASE | re.MULTILINE)

    match = pattern.search(full_text)

    if not match:
        # Coba cari tanpa anchor akhir ($) jika judulnya ada di baris yang sama dengan teks lain
        pattern_fallback = re.compile(r'^\s*(\bREFERENCES\b|\bDAFTAR PUSTAKA\b|\bBIBLIOGRAPHY\b)',
                                      re.IGNORECASE | re.MULTILINE)
        match = pattern_fallback.search(full_text)

    if not match:
        print("ℹ 'References'/'Daftar Pustaka' section not found.")
        return []

    # Ambil semua teks SETELAH judul "REFERENCES"
    text_after = full_text[match.end():].strip()

    if not text_after:
         print("ℹ 'References' section found but was empty.")
         return []

    # Heuristik untuk memisahkan entri referensi:
    # 1. Pisahkan teks referensi menjadi baris-baris
    # 2. Asumsikan entri BARU jika baris dimulai dengan:
    #    - [angka] (misal [1], [23])
    #    - Author, (misal Ahmed, U.)
    #    - angka. (misal 1., 23.)
    # 3. Jika tidak, anggap itu adalah kelanjutan dari entri sebelumnya.

    lines = text_after.splitlines()
    references = []
    current_ref = ""

    for line in lines:
        stripped_line = line.strip()
        if not stripped_line: # Lewati baris kosong
            continue

        is_new_entry = False
        # Cek apakah ini awal dari entri baru
        if re.match(r'^\s*\[\d+\]', stripped_line): # Pola [1]
            is_new_entry = True
        elif re.match(r'^\s*[A-Z][a-z]+,', stripped_line): # Pola Author, (heuristik)
            is_new_entry = True
        elif re.match(r'^\s*\d+\.', stripped_line): # Pola 1.
             is_new_entry = True

        if is_new_entry:
            if current_ref: # Simpan referensi sebelumnya
                references.append(current_ref.strip())
            current_ref = stripped_line # Mulai referensi baru
        else:
            # Ini adalah kelanjutan dari referensi sebelumnya
            if current_ref: # Hanya tambahkan jika sudah ada current_ref
                current_ref += " " + stripped_line

    if current_ref: # Simpan referensi terakhir
        references.append(current_ref.strip())

    # Filter hasil akhir (jika ada yang terlalu pendek/aneh)
    final_references = [ref for ref in references if len(ref) > 20] # Filter referensi yg terlalu pendek

    if not final_references and text_after:
        # Fallback: Jika heuristik gagal tapi teks ada, kembalikan teks sebagai satu blok
        print("ℹ Heuristik pemisahan referensi gagal, mengembalikan sebagai blok teks.")
        return [text_after]

    return final_references
# --- AKHIR FUNGSI BARU ---


def extract_keywords_bert(text, kw_model_instance, top_n=10, ngram_range=(1, 3)):
    """Mengekstrak kata kunci, dengan memotong teks dan error handling lebih baik."""
    if not text or not isinstance(text, str) or len(text.strip()) < 50:
        return []
    try:
        truncated_text = text[:MAX_CHARS_FOR_MODEL]
        keywords = kw_model_instance.extract_keywords(truncated_text,
                                                 keyphrase_ngram_range=ngram_range,
                                                 stop_words='english',
                                                 use_maxsum=True, nr_candidates=20, top_n=top_n)
        filtered_keywords = [(kw, score) for kw, score in keywords if len(kw) > 2 and not kw.isdigit()]
        return [kw for kw, score in filtered_keywords]
    # Tangkap RuntimeError secara spesifik untuk masalah CUDA
    except RuntimeError as e:
        if "CUDA" in str(e):
            print(f"❌ Error CUDA (Runtime) saat ekstraksi KeyBERT.")
        else:
            print(f"❌ Error Runtime saat ekstraksi KeyBERT: {e}")
        return []
    except Exception as e:
        print(f"❌ Error Lain saat ekstraksi KeyBERT: {e}")
        return []

def generate_summary_bart(text, summarizer_instance):
    """Menghasilkan ringkasan, dengan memotong teks dan error handling lebih baik."""
    if not text or not isinstance(text, str) or len(text.strip()) < 100:
        return "Teks terlalu pendek untuk diringkas."
    try:
        truncated_text = text[:MAX_CHARS_FOR_MODEL]
        summary = summarizer_instance(truncated_text, max_length=150, min_length=30, do_sample=False)
        return summary[0]['summary_text']
    # Tangkap RuntimeError secara spesifik untuk masalah CUDA
    except RuntimeError as e:
        if "CUDA" in str(e):
            print(f"❌ Error CUDA (Runtime) saat peringkasan BART.")
        else:
            print(f"❌ Error Runtime saat peringkasan BART: {e}")
        return "Gagal membuat ringkasan (Runtime Error)."
    except Exception as e:
        print(f"❌ Error Lain saat peringkasan BART: {e}")
        return "Gagal membuat ringkasan (Error Lain)."

def generate_embeddings(text, embedding_model_instance):
    """Menghasilkan embedding, dengan memotong teks dan error handling lebih baik."""
    if not text or not isinstance(text, str) or len(text.strip()) < 50:
        return None
    try:
        # Kita potong lebih panjang untuk embedding, tapi tetap berisiko
        truncated_text = text[:MAX_CHARS_FOR_MODEL * 2] # Batas 6000 char
        embedding = embedding_model_instance.encode(truncated_text, convert_to_numpy=True)
        return embedding
    # Tangkap RuntimeError secara spesifik untuk masalah CUDA
    except RuntimeError as e:
        if "CUDA" in str(e):
            print(f"❌ Error CUDA (Runtime) saat membuat embedding.")
        else:
            print(f"❌ Error Runtime saat membuat embedding: {e}")
        return None
    except Exception as e:
        print(f"❌ Error Lain saat membuat embedding: {e}")
        return None

print("✅ Fungsi helper (dengan download punkt paksa di split_into_sentences dan extract_references) siap.")

✅ Fungsi helper (dengan download punkt paksa di split_into_sentences dan extract_references) siap.


In [ ]:
print("⏳ Memuat model KeyBERT (all-MiniLM-L6-v2)...")
# Menggunakan model standar yang bagus untuk semantic tasks
kw_model = KeyBERT(model='all-MiniLM-L6-v2')
print("✅ Model KeyBERT berhasil dimuat.")

print("⏳ Memuat model Peringkasan (BART - distilbart-cnn-12-6)...")
# Model ini bagus untuk ringkasan berita/artikel umum, cukup ringan
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=DEVICE)
print("✅ Model Peringkasan berhasil dimuat.")

print("⏳ Memuat model Embedding (all-MiniLM-L6-v2)...")
# Menggunakan model yang sama dengan KeyBERT untuk efisiensi & konsistensi embedding
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda' if DEVICE == 0 else 'cpu')
print("✅ Model Embedding berhasil dimuat.")

⏳ Memuat model KeyBERT (all-MiniLM-L6-v2)...
✅ Model KeyBERT berhasil dimuat.
⏳ Memuat model Peringkasan (BART - distilbart-cnn-12-6)...


Device set to use cuda:0


✅ Model Peringkasan berhasil dimuat.
⏳ Memuat model Embedding (all-MiniLM-L6-v2)...
✅ Model Embedding berhasil dimuat.


In [ ]:
print(f"🚀 Memulai Proses Inti dari folder: {DATASET_FOLDER}\n" + "="*50)

results = {} # Stores keywords, summary, text length
embeddings_map = {} # Stores filename -> embedding vector
filenames_list = [] # Stores filenames in order of processing
document_texts = {} # DICTIONARY BARU UNTUK MENYIMPAN TEKS LENGKAP

try:
    all_files = [f for f in os.listdir(DATASET_FOLDER) if f.lower().endswith(('.pdf', '.docx'))]
    total_files = len(all_files)
    if total_files == 0:
        print(f"⚠ Tidak ada file PDF atau DOCX yang ditemukan di '{DATASET_FOLDER}'.")
    else:
        print(f"🔍 Menemukan {total_files} file PDF/DOCX untuk diproses.")
except FileNotFoundError:
    print(f"❌ FATAL ERROR: Folder dataset tidak ditemukan di '{DATASET_FOLDER}'. Hentikan eksekusi.")
    all_files = []

file_counter = 0
start_time = time.time()

# --- Main Loop ---
for filename in all_files:
    file_counter += 1
    print(f"\n[{file_counter}/{total_files}]--- Processing: {filename} ---")
    file_path = os.path.join(DATASET_FOLDER, filename)

    # Initialize result entry for this file
    # TAMBAHKAN 'references': []
    results[filename] = {'text_length': 0, 'keywords': [], 'summary': '', 'references': [], 'embedding': None}

    # 1. Extract Text
    full_text = None
    start_extract = time.time()
    if filename.lower().endswith(".pdf"):
        full_text = extract_text_from_pdf(file_path)
    elif filename.lower().endswith(".docx"):
        full_text = extract_text_from_docx(file_path)
    extract_duration = time.time() - start_extract

    if full_text:
        document_texts[filename] = full_text
        text_len = len(full_text)
        results[filename]['text_length'] = text_len
        print(f"📄 Teks diekstrak ({text_len:,} chars) dalam {extract_duration:.2f} dtk.")

        # 2. Extract Keywords
        kw_start = time.time()
        keywords = extract_keywords_bert(full_text, kw_model) # Fungsi sudah diperbaiki
        results[filename]['keywords'] = keywords
        kw_duration = time.time() - kw_start
        print(f"🔑 Kata Kunci ({len(keywords)}): {keywords} ({kw_duration:.2f} dtk)")

        # 3. Generate Summary
        sum_start = time.time()
        summary_text = generate_summary_bart(full_text, summarizer) # Fungsi sudah diperbaiki
        results[filename]['summary'] = summary_text
        sum_duration = time.time() - sum_start
        # Hanya tampilkan ringkasan jika tidak gagal
        summary_display = summary_text[:150]+"..." if not summary_text.startswith("Gagal") else summary_text
        print(f"📝 Ringkasan: {summary_display} ({sum_duration:.2f} dtk)")

        # 4. Extract References (FUNGSI BARU)
        ref_start = time.time()
        references = extract_references(full_text) # Panggil fungsi baru
        results[filename]['references'] = references # Simpan hasilnya
        ref_duration = time.time() - ref_start
        print(f"📚 Referensi diekstrak ({len(references)} buah) ({ref_duration:.2f} dtk)")
        # Tampilkan 1 referensi pertama sebagai sampel
        if references:
            print(f"   [Contoh Ref]: {references[0][:100]}...")

        # 5. Generate Embedding (sebelumnya #4)
        emb_start = time.time()
        embedding_vector = generate_embeddings(full_text, embedding_model) # Fungsi sudah diperbaiki
        emb_duration = time.time() - emb_start
        if embedding_vector is not None:
            embeddings_map[filename] = embedding_vector
            filenames_list.append(filename)
            # Simpan sebagai list hanya jika berhasil
            results[filename]['embedding'] = embedding_vector.tolist()
            print(f"🧠 Embedding dibuat (dim: {embedding_vector.shape[0]}) ({emb_duration:.2f} dtk)")
        else:
            # Pastikan embedding disimpan sebagai None jika gagal
            results[filename]['embedding'] = None
            print(f"❌ Embedding Gagal.")

    else:
        print(f"❌ Ekstraksi Teks Gagal.")
        results[filename]['error'] = 'Text extraction failed'
        results[filename]['embedding'] = None # Pastikan None jika teks gagal


# --- Finalization ---
end_time = time.time()
print("\n" + "="*50 + f"\n✅ Proses Ekstraksi & Analisis Inti Selesai!")
print(f"⏱ Total Waktu: {end_time - start_time:.2f} detik")
print(f"📊 Jumlah File Diproses: {file_counter}")
print(f"📈 Jumlah File dengan Embedding: {len(embeddings_map)}")

# --- Save Results ---
print(f"\n💾 Menyimpan hasil detail ke: {RESULTS_JSON_PATH}")
try:
    # Convert numpy arrays in embeddings_map to lists for JSON serialization
    results_to_save = {}
    for fname, data in results.items():
        results_to_save[fname] = data.copy() # Avoid modifying original dict
        if fname in embeddings_map:
            # Check if embedding exists and is numpy array before converting
            if embeddings_map[fname] is not None and isinstance(embeddings_map[fname], np.ndarray):
                results_to_save[fname]['embedding'] = embeddings_map[fname].tolist()
            else:
                results_to_save[fname]['embedding'] = None # Store None if embedding failed
        else:
            results_to_save[fname]['embedding'] = None # Handle cases where embedding might not exist

    with open(RESULTS_JSON_PATH, 'w') as f:
        json.dump(results_to_save, f, indent=2)
    print(f"✅ Hasil berhasil disimpan.")
except Exception as e:
    print(f"❌ Gagal menyimpan hasil ke JSON: {e}")

🚀 Memulai Proses Inti dari folder: /content/drive/MyDrive/paper_TA
🔍 Menemukan 11 file PDF/DOCX untuk diproses.

[1/11]--- Processing: Using Financial News Sentiment for Stock Price Direction Prediction.pdf ---
📄 Teks diekstrak (60,129 chars) dalam 2.62 dtk.
🔑 Kata Kunci (10): ['sentiment information analysis', 'prediction keywords sentiment', 'sentiment scores titles', 'predicted sentiment scores', 'sentiment information texts', 'sentiment classification', 'financial market sentiment', 'news sentiment stock', 'extract market sentiment', 'financial news sentiment'] (5.12 dtk)
📝 Ringkasan:  Using sentiment information in the analysis of financial markets has attracted much attention . Natural language processing methods can be used to ex... (0.74 dtk)
📚 Referensi diekstrak (30 buah) (0.00 dtk)
   [Contoh Ref]: 1. Nassirtoussi, A.K.; Aghabozorgi, S.; Wah, T.Y.; Ngo, D.C.L. Text mining for market prediction: A ...
🧠 Embedding dibuat (dim: 384) (0.01 dtk)

[2/11]--- Processing: Deep Learni

In [ ]:
print("\n" + "="*50 + "\n🔗 Memulai Analisis Hubungan Antar Dokumen...")

# Prepare matrix only from successfully generated embeddings
valid_embeddings = [embeddings_map[fname] for fname in filenames_list if embeddings_map.get(fname) is not None]
valid_filenames = [fname for fname in filenames_list if embeddings_map.get(fname) is not None]

if len(valid_embeddings) >= 2:
    embedding_matrix = np.array(valid_embeddings)
    print(f"📐 Menghitung Cosine Similarity untuk {len(valid_filenames)} dokumen...")
    start_sim_time = time.time()
    similarity_matrix = cosine_similarity(embedding_matrix)
    end_sim_time = time.time()
    print(f"✅ Matriks Kemiripan ({similarity_matrix.shape}) selesai ({end_sim_time - start_sim_time:.2f} dtk).")
    print("\n--- Matriks Kemiripan (Nilai Sebenarnya) ---")
    print(similarity_matrix.round(3))

    # --- Prepare Graph Data ---
    nodes = []
    edges = []
    print(f"\n📈 Membangun data graf (Threshold > {SIMILARITY_THRESHOLD})...")
    for i in range(len(valid_filenames)):
        fname_i = valid_filenames[i]
        nodes.append({"id": fname_i, "label": fname_i[:25]+"..."}) # Slightly longer label

        for j in range(i + 1, len(valid_filenames)):
            fname_j = valid_filenames[j]
            similarity_score = similarity_matrix[i][j]
            if similarity_score > SIMILARITY_THRESHOLD:
                edges.append({
                    "from": fname_i, "to": fname_j,
                    "value": float(similarity_score),
                    "title": f"Similarity: {similarity_score:.3f}"
                })

    print(f"📊 Data Graf Siap: {len(nodes)} nodes, {len(edges)} edges.")

    # --- Save Graph Data ---
    graph_data = {"nodes": nodes, "edges": edges}
    print(f"\n💾 Menyimpan data graf ke: {GRAPH_JSON_PATH}")
    try:
        with open(GRAPH_JSON_PATH, 'w') as f:
            json.dump(graph_data, f, indent=2)
        print(f"✅ Data graf berhasil disimpan.")
    except Exception as e:
        print(f"❌ Gagal menyimpan data graf: {e}")

    # --- Show Strongest Relationships ---
    if edges:
        sorted_edges = sorted(edges, key=lambda x: x['value'], reverse=True)
        print("\n✨ Contoh 5 Hubungan Terkuat:")
        for edge in sorted_edges[:5]:
            print(f" g  - {edge['from'][:30]}... <-> {edge['to'][:30]}... (Sim: {edge['value']:.3f})")
    else:
        print("ℹ Tidak ditemukan hubungan signifikan di atas threshold.")

else:
    print("⚠ Jumlah dokumen dengan embedding valid kurang dari 2. Analisis hubungan tidak dapat dilakukan.")

print("\n" + "="*50 + "\n✅ Analisis Hubungan Selesai!")


🔗 Memulai Analisis Hubungan Antar Dokumen...
📐 Menghitung Cosine Similarity untuk 11 dokumen...
✅ Matriks Kemiripan ((11, 11)) selesai (0.00 dtk).

--- Matriks Kemiripan (Nilai Sebenarnya) ---
[[1.    0.562 0.671 0.531 0.658 0.502 0.705 0.639 0.617 0.589 0.526]
 [0.562 1.    0.521 0.679 0.638 0.324 0.501 0.519 0.539 0.435 0.357]
 [0.671 0.521 1.    0.443 0.54  0.606 0.664 0.697 0.715 0.503 0.495]
 [0.531 0.679 0.443 1.    0.701 0.3   0.417 0.503 0.51  0.339 0.286]
 [0.658 0.638 0.54  0.701 1.    0.458 0.591 0.614 0.589 0.516 0.464]
 [0.502 0.324 0.606 0.3   0.458 1.    0.523 0.539 0.508 0.514 0.558]
 [0.705 0.501 0.664 0.417 0.591 0.523 1.    0.729 0.708 0.707 0.532]
 [0.639 0.519 0.697 0.503 0.614 0.539 0.729 1.    0.784 0.628 0.495]
 [0.617 0.539 0.715 0.51  0.589 0.508 0.708 0.784 1.    0.654 0.566]
 [0.589 0.435 0.503 0.339 0.516 0.514 0.707 0.628 0.654 1.    0.572]
 [0.526 0.357 0.495 0.286 0.464 0.558 0.532 0.495 0.566 0.572 1.   ]]

📈 Membangun data graf (Threshold > 0.5)...
📊 

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
import itertools

print("\n" + "="*50 + "\n💡 Memulai Inovasi 2: Identifikasi Celah Riset Terpersonalisasi...")

# --- Memeriksa Ketersediaan Data ---
required_vars_inno2 = ['embeddings_map', 'filenames_list', 'results']
vars_exist_inno2 = True
for var in required_vars_inno2:
    if var not in globals():
        print(f"❌ FATAL ERROR: Variabel '{var}' belum terdefinisi. Pastikan Cell 5 sudah dijalankan.")
        vars_exist_inno2 = False
if not vars_exist_inno2:
    raise NameError("Variabel penting untuk Inovasi 2 tidak ditemukan.")

# Filter hanya file yang berhasil di-embed
valid_filenames_inno2 = [fname for fname in filenames_list if embeddings_map.get(fname) is not None]
valid_embeddings_inno2 = [embeddings_map[fname] for fname in valid_filenames_inno2 if embeddings_map.get(fname) is not None]

if len(valid_embeddings_inno2) < 3: # Butuh setidaknya 3 dokumen untuk clustering yang berarti
    print(f"⚠ Jumlah dokumen dengan embedding valid ({len(valid_embeddings_inno2)}) terlalu sedikit untuk analisis kluster yang bermakna.")
else:
    print(f"✅ Menemukan {len(valid_embeddings_inno2)} dokumen dengan embedding valid untuk dianalisis klusternya.")
    embedding_matrix_inno2 = np.array(valid_embeddings_inno2)

    # --- Langkah 1: Klasterisasi Dokumen (K-Means) ---
    # Menentukan jumlah kluster (heuristic sederhana)
    # Target: sekitar 2-5 kluster, tapi tidak lebih dari jumlah dokumen / 2
    num_documents = embedding_matrix_inno2.shape[0]
    n_clusters = max(2, min(5, num_documents // 2))
    print(f"\n⏳ Melakukan klasterisasi K-Means dengan k={n_clusters}...")
    start_cluster_time = time.time()
    try:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10) # n_init='auto' atau 10
        cluster_labels = kmeans.fit_predict(embedding_matrix_inno2)
        end_cluster_time = time.time()
        print(f"✅ Klasterisasi selesai ({end_cluster_time - start_cluster_time:.2f} dtk).")

        # --- Langkah 2: Analisis Konten Kluster (Menggunakan Kata Kunci) ---
        print("\n📊 Menganalisis konten kluster berdasarkan kata kunci...")
        clusters = {i: [] for i in range(n_clusters)} # {cluster_id: [filename1, filename2,...]}
        cluster_keywords = {i: Counter() for i in range(n_clusters)} # {cluster_id: Counter(keyword: count)}

        for filename, label in zip(valid_filenames_inno2, cluster_labels):
            clusters[label].append(filename)
            # Ambil kata kunci dari hasil Cell 5
            if filename in results and 'keywords' in results[filename]:
                # Update counter kata kunci untuk kluster ini
                cluster_keywords[label].update(results[filename]['keywords'])

        # Tampilkan hasil klasterisasi dan kata kunci dominan
        print("\n--- Hasil Klasterisasi Dokumen ---")
        for i in range(n_clusters):
            print(f"\n  Cluster {i}:")
            print(f"    Dokumen ({len(clusters[i])}):")
            for fname in clusters[i]:
                print(f"      - {fname}")
            # Tampilkan 5 kata kunci paling umum di kluster ini
            top_5_keywords = cluster_keywords[i].most_common(5)
            print(f"    Top Keywords: {[kw for kw, count in top_5_keywords]}")

        # --- Langkah 3: Identifikasi Potensi Celah Riset (Heuristik Sederhana) ---
        print("\n\n💡 Mencari Potensi Celah Riset (Area Persimpangan)...")
        suggested_gaps = []
        # Cari kombinasi 2 kluster yang berbeda
        for i, j in itertools.combinations(range(n_clusters), 2):
            # Ambil 3 keyword teratas dari masing-masing kluster
            keywords_i = [kw for kw, count in cluster_keywords[i].most_common(3)]
            keywords_j = [kw for kw, count in cluster_keywords[j].most_common(3)]

            # Buat saran sederhana dengan menggabungkan keyword
            if keywords_i and keywords_j: # Pastikan ada keyword
                # Saran 1: Keyword teratas dari i + Keyword teratas dari j
                suggestion1 = f"Kombinasi antara '{keywords_i[0]}' (dari Cluster {i}) dan '{keywords_j[0]}' (dari Cluster {j})"
                suggested_gaps.append(suggestion1)
                # Saran 2 (jika keyword cukup): Keyword kedua dari i + Keyword kedua dari j
                if len(keywords_i) > 1 and len(keywords_j) > 1:
                    suggestion2 = f"Persimpangan area '{keywords_i[1]}' (Cluster {i}) dan '{keywords_j[1]}' (Cluster {j})"
                    suggested_gaps.append(suggestion2)

        if suggested_gaps:
            print("\n--- Saran Area Potensial (Berdasarkan Koleksi Anda Saat Ini) ---")
            for idx, gap in enumerate(suggested_gaps[:5]): # Tampilkan maksimal 5 saran
                print(f"  {idx+1}. {gap}")
            print("\n(Saran ini bersifat heuristik, perlu validasi lebih lanjut)")
        else:
            print("ℹ Tidak dapat menghasilkan saran celah riset otomatis dari kluster yang ada.")

    except Exception as cluster_error:
        print(f"❌ Error saat melakukan klasterisasi atau analisis: {cluster_error}")

print("\n" + "="*50 + "\n✅ Inovasi 2 Selesai!")


💡 Memulai Inovasi 2: Identifikasi Celah Riset Terpersonalisasi...
✅ Menemukan 11 dokumen dengan embedding valid untuk dianalisis klusternya.

⏳ Melakukan klasterisasi K-Means dengan k=5...
✅ Klasterisasi selesai (0.03 dtk).

📊 Menganalisis konten kluster berdasarkan kata kunci...

--- Hasil Klasterisasi Dokumen ---

  Cluster 0:
    Dokumen (3):
      - Deep Learning for Aspect-Based Sentiment Analysis A Comparative Review.pdf
      - Aspect-Based_Sentiment_Analysis_of_Twitter_Influencers_to_Predict_the_Trend_of_Cryptocurrencies_Based_on_Hybrid_Deep_Transfer_Learning_Models.pdf
      - Forecasting directional bitcoin price returns using aspect-based sentiment analysis on online text data.pdf
    Top Keywords: ['aspect based sentiment', 'content web sentiment', 'reviews sentiment classification', 'ences sentiment analysis', 'tweets deep learning']

  Cluster 1:
    Dokumen (3):
      - Using Financial News Sentiment for Stock Price Direction Prediction.pdf
      - An Entropy-based Eval